## Import Requirements and Datasets

In [1]:
# requirements
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
# dataset 1
df_data = pd.read_csv('original_datasets/data.csv')
df_data.head()

,Date,Selina,Sub Category,Capacity,Out of Order,Available Beds,Sold Beds,Room Revenue USD,Food and Beverage Revenue,Activities Revenue,Co-Working Revenue,Average Guest Review
0,2017-12-01,San Jose,Dorm,86,1,85,65,1170.0,341.640,179.010,79.560,8.755
1,2017-12-01,Medellin,Dorm,215,25,190,91,1820.0,373.100,274.820,72.800,8.787
2,2017-12-01,Jaco,Dorm,118,2,116,55,880.0,289.520,100.320,23.760,8.780
3,2017-12-01,Bocas del Toro,Dorm,154,2,152,108,1728.0,618.624,390.528,164.160,8.689
4,2017-12-01,Red Frog,Dorm,24,1,23,17,255.0,91.545,44.880,20.655,8.754


In [3]:
# dataset 2
df_op_dates = pd.read_csv('original_datasets/opening_dates.csv')
df_op_dates.head()

,Selina,Selina Country,Selina Opening Date
0,Venao,Panama,2014-08-01
1,Bocas del Toro,Panama,2014-12-01
2,Pedasi,Panama,2016-03-01
3,San Jose,Costa Rica,2016-11-01
4,Red Frog,Panama,2016-11-01


## Assessing

### Visual Assessment

#### List of Visual Assessments:

- Contains Selinas that opened outside of Q4 2017-2018 date range (ref. opening dates):
    - Are all Selina's listed considered "newly opened" and we're simply analyzing their 2018 performance?
    - Or do we remove all Selinas that opened prior to the data collection period?
- Missing columns:
    - Total revenue
    - Selina's KPIs
- Some Selina's are missing chunks of data for certain bed types and some are missing individual rows of data for certain bed types:
    - Does this mean Selina didn't have this bed type available?
    - Or maybe this bed type wasn't booked for that day?
- Column names not in snake_case
- Missing categorical columns that could potentially add value
    - Country
    - Month of year each Selina opened

### Programmatic Assessment

In [4]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19077 entries, 0 to 19076
Data columns (total 12 columns):
Date                         19077 non-null object
Selina                       19077 non-null object
Sub Category                 19077 non-null object
Capacity                     19077 non-null int64
Out of Order                 19077 non-null int64
Available Beds               19077 non-null int64
Sold Beds                    19077 non-null int64
Room Revenue USD             19077 non-null float64
Food and Beverage Revenue    19077 non-null float64
Activities Revenue           19077 non-null float64
Co-Working Revenue           19077 non-null float64
Average Guest Review         19077 non-null float64
dtypes: float64(5), int64(4), object(3)
memory usage: 1.7+ MB


In [5]:
df_data.duplicated().sum()

0

In [6]:
df_data.describe()

,Capacity,Out of Order,Available Beds,Sold Beds,Room Revenue USD,Food and Beverage Revenue,Activities Revenue,Co-Working Revenue,Average Guest Review
count,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000,19077.000000
mean,64.614195,4.362426,60.251769,31.359228,629.480657,151.040265,92.938102,32.068546,8.778343
std,53.954401,5.809417,49.804572,25.606965,411.518606,134.996453,74.167449,29.188877,0.091046
min,6.000000,0.000000,5.000000,2.000000,36.000000,0.000000,0.000000,0.000000,8.685000
25%,24.000000,1.000000,23.000000,13.000000,324.000000,57.214350,42.882840,13.547520,8.717000
50%,40.000000,1.000000,39.000000,22.000000,540.000000,120.669120,72.421020,23.332608,8.753000
75%,96.000000,5.000000,86.000000,41.000000,820.800000,207.820800,122.068800,41.164200,8.785000
max,254.000000,43.000000,233.000000,126.000000,2541.000000,900.191250,541.479920,277.397500,9.010000


In [7]:
# check if all df_data Selinas are in df_op_dates DataFrame, or if any Selinas need to be removed from df_data
print(len(df_data.Selina.unique()))

count = 0

for selina in df_data.Selina.unique():
    if selina in df_op_dates.Selina.unique():
        count += 1
    else:
        count += 0

print(count)

23
23


#### List of Programmatic Assessments:
- No NaNs
    - If we add rows for the missing data recognized in the visual assessment, we will create NaNs that need to be filled accordingly
- Date column wrong dtype
- No duplicates

## Cleaning

### List of Cleaning Issues:

1. Convert column names to snake_case.
2. Convert date column to datetime dtype.
3. Create new columns for total_revenue and the missing KPIs of occupancy_rate and trevpob.
4. Create new categorical columns for country and month_of_opening (month of the year that the Selina opened).
5. Create new rows for the missing data for each room type at each Selina by separating into separate DataFrames by Selina.
6. Fill in NaNs from newly added rows in each separate DataFrame.
7. Merge the separate DataFrames back into a master dataset now that all rows have been added and NaNs filled.

### Issue 1:

#### Define:
Convert column names to snake_case using .replace() in order to make Python functions and methods more efficient.

#### Code:

In [8]:
df_data.columns = df_data.columns.str.lower().str.replace(" ","_").str.replace("-","_")

#### Test:

In [9]:
df_data.columns

Index(['date', 'selina', 'sub_category', 'capacity', 'out_of_order',
       'available_beds', 'sold_beds', 'room_revenue_usd',
       'food_and_beverage_revenue', 'activities_revenue', 'co_working_revenue',
       'average_guest_review'],
      dtype='object')

### Issue 2:

#### Define:
Convert date column to datetime dtype.

#### Code:

In [10]:
df_data['date'] = pd.to_datetime(df_data['date'])

#### Test:

In [11]:
df_data.date.dtypes

dtype('<M8[ns]')

### Issue 3:

#### Define:
Create new columns for total_revenue, occupancy_rate, and trevpob.

#### Code:

In [ ]:
#df_data['total_revenue'] = df_data.apply(lambda x: x.room_revenue_usd + x.food_and_beverage_revenue + x.activities_revenue + x.co_working_revenue, axis=1)
#df_data['trevpob'] = df_data.total_revenue / df_data.
#df_data['occupancy_rate'] = 